<a href="https://colab.research.google.com/github/isamu-isozaki/toxic-joke-generator/blob/master/GPT_Jokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Thanks https://www.reddit.com/r/MachineLearning/comments/bgvzdu/d_jokes_generated_via_gpt2/
import os
import json
import random
import re

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
folder_id = []

In [0]:
def ListFolder(parent, show=False):
  global folder_id
  filelist=[]
  file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
  for f in file_list:
    if show:
      print(f"{f['title']}, {f['id']}")
    if f['mimeType']=='application/vnd.google-apps.folder': # if folder
        if f['title'] == "autoencoder1":
          folder_id = f['id']
          filelist.append({"id":f['id'],"title":f['title'], "mimeType": 'application/vnd.google-apps.folder', "list":ListFolder(f['id'])})
        else:
          for f1 in ListFolder(f['id']):
            filelist.append(f1)
    else:
        filelist.append({"id":f['id'],"title":f['title'],"mimeType":"file","title1":f['alternateLink']})
  return filelist
fl = ListFolder('root')

In [0]:
for f in fl:
  if (f['title'] == "gpt-2.zip") and os.path.isfile(f['title']) == False:
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    fil = drive.CreateFile({'id':f['id']})
    fil.GetContentFile(f['title'])

In [0]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('gpt-2.zip', 'r') as zipObj:
    zipObj.extractall()

In [42]:
cd gpt-2

/content/gpt-2


In [0]:
if not os.path.exists('checkpoint'):
  os.makedirs("checkpoint")

In [44]:
cd checkpoint

/content/gpt-2/checkpoint


In [0]:
if not os.path.exists('run1'):
  os.makedirs("run1")

In [0]:
for f in fl:
  if f['title'] == "autoencoder1":
    for part in f["list"]:
      fil = drive.CreateFile({'id':part['id']})
      fil.GetContentFile(part['title'][18:])
      os.rename(part['title'][18:], "./run1/"+part['title'][18:])

In [49]:
cd ..

/content/gpt-2


In [15]:
!pip3 install -r requirements.txt

In [0]:
!python ./train.py --dataset jokes.txt.npz --batch_size 2 --sample_every 100 --save_every 1000 --folder_id $folder_id

2019-05-03 11:17:53.532871: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-03 11:17:53.533131: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x75ae6e0 executing computations on platform Host. Devices:
2019-05-03 11:17:53.533164: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-03 11:17:53.709250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-03 11:17:53.709769: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x75ef440 executing computations on platform CUDA. Devices:
2019-05-03 11:17:53.709797: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-03 11:17:53.710161: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [77]:
!python3 generate_unconditional_samples.py --top_k 40 --temperature 1.2

Exception ignored in: <function _get_module_lock.<locals>.cb at 0x7fa51b457158>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 176, in cb
KeyboardInterrupt: 
Traceback (most recent call last):
  File "generate_unconditional_samples.py", line 9, in <module>
    import model, sample, encoder
  File "/content/gpt-2/model.py", line 3, in <module>
    from tensorflow.contrib.training import HParams
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/__init__.py", line 37, in <module>
    from tensorflow.contrib import cudnn_rnn
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/cudnn_rnn/__init__.py", line 38, in <module>
    from tensorflow.contrib.cudnn_rnn.python.layers import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/cudnn_rnn/python/layers/__init__.py", line 23, in <module>
    from tensorflow.contrib.cudnn_rnn.python.layers.cudnn_rnn import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflo